In [1]:
import pandas as pd
import statsmodels.api as sm

In [2]:
SP = pd.read_csv('MSCI.csv', parse_dates=True, index_col='Date',)
SP

,Price MSCI,Price GN,Price Demant,Price Sonova
Date,,,,
2023-01-04,"2.829,20","152,8","260,4","282,5"
2023-01-03,"2.791,44","153,9","240,2","268,6"
2023-01-02,"2.714,57","152,4","210,9","231,2"
2023-01-01,"2.785,00","167,8","192,85","227,9"
2022-01-12,"2.602,69","159,75","192,55","219,3"
...,...,...,...,...
2018-01-08,"2.175,50","333,1","261,2","182,68"
2018-01-07,"2.153,10",304,"304,6","181,59"
2018-01-06,"2.089,30","290,8","256,8","176,67"


In [3]:
SP = SP[SP.columns[SP.columns.isin(['Date','Price Sonova', 'Price MSCI'])]]
SP['Price Sonova'] = SP['Price Sonova'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
SP['Price MSCI'] = SP['Price MSCI'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
SP

/tmp/ipykernel_3000/3441546724.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['Price Sonova'] = SP['Price Sonova'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)
/tmp/ipykernel_3000/3441546724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP['Price MSCI'] = SP['Price MSCI'].str.replace('.','', regex=True).str.replace(',', '.', regex=True).astype(float)


,Price MSCI,Price Sonova
Date,,
2023-01-04,2829.20,282.50
2023-01-03,2791.44,268.60
2023-01-02,2714.57,231.20
2023-01-01,2785.00,227.90
2022-01-12,2602.69,219.30
...,...,...
2018-01-08,2175.50,182.68
2018-01-07,2153.10,181.59
2018-01-06,2089.30,176.67


In [4]:
SP.sort_values(by='Date', inplace = True) 
SP

/tmp/ipykernel_3000/1494837875.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SP.sort_values(by='Date', inplace = True)


,Price MSCI,Price Sonova
Date,,
2018-01-04,2086.51,163.11
2018-01-05,2092.92,170.86
2018-01-06,2089.30,176.67
2018-01-07,2153.10,181.59
2018-01-08,2175.50,182.68
...,...,...
2022-01-12,2602.69,219.30
2023-01-01,2785.00,227.90
2023-01-02,2714.57,231.20


In [5]:
daily_ret = SP.pct_change(1)
daily_ret_clean = daily_ret.dropna(axis=0)
daily_ret_clean.head()

,Price MSCI,Price Sonova
Date,,
2018-01-05,0.003072,0.047514
2018-01-06,-0.001730,0.034004
2018-01-07,0.030537,0.027849
2018-01-08,0.010404,0.006003
2018-01-09,0.003912,0.062021


In [6]:

# split dependent and independent variable
X = daily_ret_clean['Price MSCI']
y = daily_ret_clean['Price Sonova']

# Add a constant to the independent value
X1 = sm.add_constant(X)

# make regression model 
model = sm.OLS(y, X1)

# fit model and print results
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           Price Sonova   R-squared:                       0.339
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     29.78
Date:                Wed, 19 Apr 2023   Prob (F-statistic):           1.05e-06
Time:                        12:13:15   Log-Likelihood:                 72.664
No. Observations:                  60   AIC:                            -141.3
Df Residuals:                      58   BIC:                            -137.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0069      0.010      0.719      0.4

In [7]:
from scipy import stats
slope, intercept, r_value, p_value, std_err = stats.linregress(X, y)

print(f'Beta = {slope}')

Beta = 0.9949147033651533
